In [0]:
%sh
mkdir -p /databricks/otel && 
wget -P /databricks/otel https://github.com/christophergrant/databricks-opentelemetry/releases/download/0.0.1/databricks-otelcol-amd64.zip 

In [0]:
%sh
unzip /databricks/otel/databricks-otelcol-amd64.zip -d /databricks/otel && chmod +x /databricks/otel/databricks-otelcol

In [0]:
%sh

# Define the output file name
CONFIG_FILE="otelcol-config.yaml"

# Write the configuration to the file
cat << 'EOF' > /databricks/otel/$CONFIG_FILE
receivers:
  hostmetrics:
    collection_interval: 1m
    scrapers:
      cpu:
      memory:
      disk:
      load:
      filesystem:
      network:

exporters:
  prometheus:
    endpoint: "0.0.0.0:9090"

processors:

service:
  pipelines:
    metrics:
      receivers: [hostmetrics]
      exporters: [prometheus]
      processors: []
EOF

# Print a message indicating completion
echo "Configuration has been written to $CONFIG_FILE"


In [0]:
%sh
nohup /databricks/otel/databricks-otelcol --config /databricks/otel/otelcol-config.yaml > /databricks/otel/otelcol.log 2>&1 &


In [0]:
import requests
import time

time.sleep(15)

response = requests.get('http://localhost:9090/metrics')
response.raise_for_status()

num_lines = len(response.text.splitlines())
assert num_lines > 10, f"Expected more than 10 lines, got {num_lines}"